# 8.3 时间序列分析

时序数据 (time series) 是按时间顺序排列的一系列数值。金融时间序列具有高噪声、非平稳、非线性、混沌、潜在周期性等特点（Yaser，1996）。

本节将介绍时间序列模型的以下方面，这些方面将在后续案例研究中加以利用。



## 1. 时间序列的分解

时间序列可以分解为以下成分：

* **趋势成分**

趋势是指时间序列中持续的方向性运动。趋势可以分为确定性趋势和随机趋势。确定性趋势可以为我们提供解释趋势的潜在原因，而随机趋势则是无法解释的序列随机特性。趋势经常出现在金融序列中，许多交易模型都使用复杂的趋势识别算法。

* **季节成分**

许多时间序列包含季节性变化。这在代表商业销售或气候水平的序列中尤为常见。在量化金融中，我们经常看到季节性变化，特别是与假日季节或年度温度变化（例如天然气）相关的序列。


我们可以吧一条时间序列$y_t$写成：

$$
y_t = S_t + T_t + R_t
$$

这里$S_t$是季节成分，$T_t$是趋势成分，$R_t$代表了其他没有被前两者捕获的成分。

如果用Python代码表示的话：

```python
import statsmodels.api as sm 
sm.tsa.seasonal_decompose(Y,freq=52).plot()
```

下图展示了分解成趋势、季节性和剩余成分的时间序列。将时间序列分解成这些成分可以帮助我们更好地理解时间序列并识别其行为，从而做出更好的预测。

<img src='images/tsc.png' width=800>
图：时间序列的分解



底部三个子图分别显示了三个时间序列成分。这些成分可以相加重建顶部子图中显示的实际时间序列（标为“观测值”）。请注意，时间序列在 2017 年之后呈现趋势成分。因此，该时间序列的预测模型应该包含 2017 年之后趋势行为的信息。

在季节性方面，年初时会出现一些幅度的增加。底部子图中显示的残差成分是在从数据中减去季节性和趋势成分后剩下的部分。残差成分大部分是平坦的，在 2018 年和 2019 年左右有一些尖峰和噪声。此外，每个图的刻度都不一样，趋势成分的范围最大，如其图上的刻度所示。


## 2. 时间序列的特性

虽然将时间序列分解成趋势、季节性和残差成分相对简单，但在处理金融时间序列数据时，还有一些其他需要考虑的方面。

### 2.1 自相关 (Autocorrelation)

在许多情况下，时间序列的连续元素之间存在相关性。也就是说，序列中连续点的行为会相互影响。**自相关**是观测值之间相似性的度量，取决于它们之间的时间滞后。这种关系可以通过 **自回归模型** 进行建模。**自回归**一词表示它是变量自身与其滞后值的回归。

在自回归模型中，我们使用变量的过去值的线性组合来预测目标变量。

因此，一个 p 阶的自回归模型可以写成如下形式：


$$
Y_t = \alpha + \beta_1 * Y_{t-1} + \beta_2 * Y_{t-2} + \dots + \beta_p * Y_{t-p} + \epsilon_t
$$


其中 $\epsilon_t$ 是白噪声。**自回归模型**类似于多元回归，但使用滞后的 Y 值作为预测变量。我们将其称为 AR(p) 模型，即 p 阶的自回归模型。自回归模型在处理各种不同的时间序列模式方面非常灵活。

### 2.2 平稳性 (Stationarity)

如果时间序列的统计特性随时间推移保持不变，则称其为 **平稳** 的。因此，具有趋势或季节性的时间序列是非平稳的，因为趋势和季节性会影响时间序列的值。

例如，考虑以下图表：


<img src='images/Nonstationaryplots.png' width=600>
图：非平稳性的序列

在第一幅图中，我们可以清楚地看到平均值随着时间变化而增加 (上升)，形成上升趋势。因此，这是一个非平稳序列。要使序列被归类为平稳，它不应该表现出趋势。

接下来看第二幅图，我们确实没有看到序列中的趋势，但是序列的方差却随时间变化。平稳序列必须具有恒定的方差；因此，该序列也是非平稳序列。

在第三幅图中，随着时间的推移，传播更接近，这意味着协方差是时间的函数。图 5-2 中显示的三个示例代表了非平稳时间序列。现在再来看图 5-3 中展示的第四个图。


<img src='images/Stationaryplot.png' width=400>
图：平稳的时间序列

本例中，均值、方差和协方差随时间保持恒定。这便是平稳时间序列的特征。使用此图 (第四幅图) 预测未来值会更加容易。大多数统计模型都要求序列平稳才能进行有效和准确的预测。

如图 5-2 所示，时间序列非平稳的两个主要原因是趋势和季节性。为了使用时间序列预测模型，我们通常会将非平稳序列转换为平稳序列，因为其统计特性不会随时间变化，因此更容易建模。


### 2.3 差分
差分是使时间序列平稳的一种常用方法。 这种方法通过计算序列中连续项之间的差值来实现。 差分通常用于消除变动的均值。 数学上，差分可以表示为：


$$
y^{'} = y_t - y_{t-1}
$$

其中 $y_t$是第t期的值。

当差分后的序列为白噪声时，原始序列被称为**一阶非平稳序列**。

## 3. 传统时间序列模型


有许多建模时间序列以进行预测的方法。大多数时间序列模型旨在同时考虑趋势、季节性和组成部分，同时解决时间序列中嵌入的自相关性和平稳性。例如，前一节讨论的自回归（AR）模型解决了时间序列中的自相关性问题。

时间序列预测中最广泛使用的模型之一是ARIMA模型。

### 3.1 ARIMA模型

如果我们将平稳性与自回归和移动平均模型相结合（在本节后面进一步讨论），我们就得到了ARIMA模型。ARIMA是AutoRegressive Integrated Moving Average的缩写，它具有以下组成部分：

$AR(P)$

它代表自回归，即时间序列对自身的回归，正如前一节所讨论的，假设当前系列值依赖于其之前的值，并存在一些滞后（或多个滞后）。模型中的最大滞后被称为p。

$I(d)$

它代表整合的阶数。它简单地表示使序列平稳所需的差分次数。

$MA(q)$

它代表移动平均。恕不展开，简言之，它模拟了时间序列的误差；同样，假设当前误差取决于之前的误差，并存在一些滞后，这被称为q。

移动平均方程可以写成：

$$
y_t = c + \epsilon_t + \sum_{i=1}^{q} \theta_i \epsilon_{t-i} 
$$

其中：
- $y_t$ 是时间序列在时刻 $ t $ 的观测值。
- $c$ 是常数项。
- $\epsilon_t$ 是在时刻 $t $ 的误差项。
- $\theta_i$ 是移动平均项的系数 $i = 1, 2, \ldots, q $。

综合所有部分，完整的ARIMA模型可以写成：

$$
y'_t = c + \phi_1 y'_{t-1} + \ldots + \phi_p y'_{t-p} + \theta_1 \epsilon_{t-1} + \ldots + \theta_q \epsilon_{t-q} + \epsilon_t
$$


在这个方程中，$y'$ 是差分序列（可能已经被差分多次）位于等号左边。右边的预测变量包括 $y'$ 的滞后值和滞后误差。我们将其称为ARIMA($p,d,q$)模型，其中：
- $ p $ 是自回归部分的阶数，
- $ d $ 是差分的次数，
- $ q $ 是移动平均的阶数。

与自回归和移动平均模型使用的相同的平稳性和可逆性条件也适用于ARIMA模型。

以下显示了拟合ARIMA模型的Python代码，其阶数为(1,0,0)：

```python
from statsmodels.tsa.arima.model import ARIMA

# Fit the ARIMA model
model = ARIMA(endog=Y_train, order=[1,0,0])
fit_model = model.fit()

# Print the summary of the model
print(fit_model.summary())
```


ARIMA模型族有几个变种，其中一些如下所示：

***ARIMAX***

包含外生变量的ARIMA模型。我们将在案例研究中使用此模型。

***SARIMA***

该模型中的“S”代表季节性，旨在建模嵌入在时间序列中的季节性组成部分，以及其他组成部分。

***VARMA***

这是模型拓展到多变量情况的版本，在需要同时预测多个变量时使用。

## 4. 应用ARIMA模型预测股票价格

In [24]:
X_train_ARIMA=X_train.loc[:, ['GOOGL', 'IBM', 'SPX', 'DJI', 'VIX']]
X_test_ARIMA=X_test.loc[:, ['GOOGL', 'IBM', 'SPX', 'DJI', 'VIX']]
tr_len = len(X_train_ARIMA)
te_len = len(X_test_ARIMA)
to_len = len (X)

我们现在配置ARIMA模型，阶数为(1,0,0)，并将独立变量用作模型中的外生变量。在使用外生变量的ARIMA模型版本称为ARIMAX模型，其中的“X”代表外生变量：

In [25]:
#Diable the warnings
import warnings
warnings.filterwarnings('ignore')

# Time series Models
from statsmodels.tsa.arima.model import ARIMA
#from statsmodels.tsa.statespace.sarimax import SARIMAX
modelARIMA=ARIMA(endog=Y_train,exog=X_train_ARIMA,order=[1,0,0])
model_fit = modelARIMA.fit()

训练ARIMA模型：

In [26]:
error_Training_ARIMA = mean_squared_error(Y_train, model_fit.fittedvalues)
predicted = model_fit.predict(start = tr_len -1 ,end = to_len -1, exog = X_test_ARIMA)[1:]
error_Test_ARIMA = mean_squared_error(Y_test,predicted)
error_Test_ARIMA

0.0012461278671702539

这个ARIMA模型的误差是合理的。


到目前为止使用的模型已经通过使用时间延迟方法嵌入了时间序列组件，其中滞后变量被包含为其中一个自变量。然而，对于基于时间序列的模型，我们不需要微软（MSFT）的滞后变量作为自变量。因此，作为第一步，我们将删除这些模型中MSFT的先前回报。我们将所有其他变量作为这些模型中的外生变量。

让我们首先准备用于ARIMA模型的数据集，只包含相关变量作为外生变量：

## 参考
- 第5章 监督学习：回归，《金融机器学习和数据科学实践》，Hariom Tatsat，O'Reilly
- [Penn State's STAT 510: Applied Time Series Analysis](https://online.stat.psu.edu/stat510/)
- [《金融时间序列分析讲义》](https://www.math.pku.edu.cn/teachers/lidf/course/atsa/atsanotes/html/_atsanotes/index.html#课程内容)
- Quantecon: https://quantecon.org/lectures/
- S. Yaser, A. Atiya. Introduction to financial forecasting[J]. Applied Intelligence, 1996, 6(3): 205-213.
